### Scraping Udemy with Selenium and Beautiful Soup


In [1]:
import re
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time 
import os
import pickle
import random
import numpy as np

###### Set up and configure the driver

In [42]:
#set up the driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.implicitly_wait(3) 

###### Scraping setup 

In [105]:
SearchURL = 'https://www.udemy.com/' # 1-160
endofresult = False
run_on_page_one = True
page_counter = 1
driver.get(SearchURL)
time.sleep(1)
course_details=[] # List of dictionaries of course details
#url_list = [f'https://www.udemy.com/courses/search/?q=data%20science&src=ukw&p={p}' for p in range(1,159)]

In [73]:
# just for the first run
search_form = driver.find_element_by_xpath('//*[@id="search-field-home"]')
search_form.send_keys('Data science')
time.sleep(2)
search_form.send_keys(Keys.RETURN)

In [65]:
def get_page_soup():
    return BeautifulSoup(driver.page_source, 'html.parser')
def get_titles(soup):
    return [x.text for x in soup.find_all('h1') ]
def get_lectures_hours_levels(soup):
    
    n,i = 0,0
    lectures = []
    hours = []
    minutes = []
    levels = []
    
    lecture_hr_lvl = [x.text for x in soup.find_all(class_='list-view-course-card--item--3JHfQ') ]
    
    while n < len(lecture_hr_lvl):
        #lectures
        lectures.append(int(lecture_hr_lvl[i].strip(' lectures')))
        # hours & minutes
        if lecture_hr_lvl[i + 1][-5] == 'hours':
            hours.append(lecture_hr_lvl[i + 1].strip(' hours'))
            minutes.append(None)
        else:
            hours.append(None)
            minutes.append(lecture_hr_lvl[i + 1].strip(' mins'))
        #levels
        levels.append(lecture_hr_lvl[i + 2])
        
        i += 7
        n += 7
        
    return lectures, hours, minutes, levels

def get_sale_prices_prices(soup):
    
    sale_prices = []
    prices = []
    
    price_strings = [x.text for x in soup.find_all(class_='list-view-course-card--price--vxruy') ]
    
    for price_string in price_strings:
        pattern = r'[\d]+\.[\d]+'
        price_list = re.findall(pattern, price_string)
        if len(price_list) == 1 :
            sale_prices.append(float(price_list[0].strip('$')))
            prices.append(float(price_list[0].strip('$')))
    
        if len(price_list) == 2:
            sale_prices.append(float(price_list[0].strip('$')))
            prices.append(float(price_list[1].strip('$')))
        
    return sale_prices, prices

def get_stars_ratings(soup):
    stars = []
    ratings = []
    rating_strings = [x.text for x in soup.find_all(class_='list-view-course-card--rating--za-yU')]
    
    for rating_string in rating_strings:
        rating_string = rating_string.strip(' ratings') 
        stars.append(rating_string[:3])
        ratings_num = (rating_string[4:-8]).replace(',','')
        ratings.append(ratings_num)
        
    return stars, ratings   

def get_course_links(soup):
    course_links = []
    soup_tags = soup.find_all(class_='course-card-list--course-card-wrapper---5ot2')
    for tag in soup_tags:
        link = tag.find('a', 'href' == True)
        course_links.append(link.get('href')) 
    return course_links
    

In [66]:
def get_course_details():
    
    soup = get_page_soup()
    
    titles = get_titles(soup)
    lectures, hours, minutes,levels = get_lectures_hours_levels(soup)
    sale_prices, prices = get_sale_prices_prices(soup)
    stars, ratings = get_stars_ratings(soup)
    course_links = get_course_links(soup)
    
    all_details = zip(*[titles, lectures, hours, minutes, levels, 
                     sale_prices, prices, stars, ratings, course_links])
    all_details = list(all_details)
    course_details.extend(all_details)
    

In [108]:
course_details=[]
while (endofresult == False):
    try:
        #get-data
        get_course_details()
    except Exception as e:
        print(e)
        print('Search failed on page:')    

    page_counter += 1
    if page_counter > 160:
        endofresult = True
        print("End of serach result")
        with open("udemy_course_links.pkl",'wb') as picklefile:
            pickle.dump(course_details,picklefile)

    if(run_on_page_one):
        selector='//*[@id="search-result-page-v3"]/div/div/div[3]/div[1]/div[1]/div/div/ul/li[6]/a/span/span'
        run_on_page_one = False
    else:
        selector = '//*[@id="search-result-page-v3"]/div/div/div[3]/div[1]/div[1]/div/div/ul/li[7]/a/span/span'

    element = driver.find_element_by_xpath(selector)
    driver.execute_script("return arguments[0].scrollIntoView();", element)
    time.sleep(3)
    element.click()
        
#     except Exception as e:
#         raise
#         print(e)
#         print('Search failed on page:')    
#         count = len(course_details)
# #         if (count % 100 == 0):
#             with open("udemy_course_links.pkl",'wb') as picklefile:
#                 pickle.dump(course_details,picklefile)
        

invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '5 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:
invalid literal for int() with base 10: '2 practi'
Search failed on page:


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="search-result-page-v3"]/div/div/div[3]/div[1]/div[1]/div/div/ul/li[7]/a/span/span"}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.5 x86_64)


In [107]:
course_details=[]
while (endofresult == False):
    try:
        #get-data
        get_course_details()


        page_counter += 1
        if page_counter> 160:
            endofresult = True
            print("End of serach result")
            with open("udemy_course_links.pkl",'wb') as picklefile:
                pickle.dump(course_details,picklefile)
        
        if(run_on_page_one):
            selector='//*[@id="search-result-page-v3"]/div/div/div[3]/div[1]/div[1]/div/div/ul/li[6]/a/span/span'
            run_on_page_one = False
        else:
            selector = '//*[@id="search-result-page-v3"]/div/div/div[3]/div[1]/div[1]/div/div/ul/li[7]/a/span/span'
        
        element = driver.find_element_by_xpath(selector)
        driver.execute_script("return arguments[0].scrollIntoView();", element)
        time.sleep(2)
        element.click()
        
    except Exception as e:
        raise
        print(e)
        print('Search failed on page:')    
        count = len(course_details)
#         if (count % 100 == 0):
#             with open("udemy_course_links.pkl",'wb') as picklefile:
#                 pickle.dump(course_details,picklefile)
        

ValueError: invalid literal for int() with base 10: '2 practi'

In [59]:
len(course_details)

0

In [89]:
course_details[0]

('Android: Using Backendless to save data in the cloud',
 61,
 None,
 '11.5 hour',
 'Beginner',
 10.99,
 19.99,
 '4.7',
 '12 ',
 '/android-using-backendless-to-save-data-in-the-cloud/')

In [138]:
# finish well
driver.quit()
os.system('killall chrome')

256

In [95]:
with open("udemy_course_links_1_to_110.pkl",'wb') as picklefile:
    pickle.dump(all_data, picklefile)

In [87]:
with open("udemy_course_links_1_to_56.pkl",'rb') as picklefile:
    old_data = pickle.load(picklefile)

In [92]:
all_data = course_details + old_data

In [111]:
pd.DataFrame(all_data)


,0,1,2,3,4,5,6,7,8,9
0,Android: Using Backendless to save data in the...,61,None,11.5 hour,Beginner,10.99,19.99,4.7,12,/android-using-backendless-to-save-data-in-the...
1,Fundraising Success: The Art & Science of Majo...,24,None,2.5 hour,Intermediate,10.99,49.99,4.4,18,/fundraising-success-the-art-science-of-major-...
2,Science of Happiness Certification: Fully Accr...,44,None,2.5 hour,Beginner,10.99,199.99,4.5,24,/positive-psychology-habits-and-happiness-cert...
3,The Science of Reiki,11,None,1 hour,All Levels,10.99,49.99,4.3,37,/the-science-of-reiki/
4,GDPR & PSD2 data protection compliance for fin...,60,None,6.5 hour,All Levels,10.99,174.99,4.7,39,/understanding-financial-malware-and-how-top-b...
5,Death By PowerPoint To Killer Presentation Via...,15,None,31,All Levels,10.99,34.99,4.2,10,/death-by-powerpoint-to-killer-presentation/
6,Goal Setting for Success: The Science of Goal ...,47,None,2 hour,All Levels,10.99,199.99,4.2,39,/science-of-goal-setting/
7,How to Read Data from a Modbus RS485 Temperatu...,15,None,1 hour,Intermediate,10.99,39.99,4.4,53,/how-to-read-data-from-a-modbus-rs485-temperat...
8,Art Therapy (based on Vedic Sciences),10,None,1 hour,All Levels,10.99,99.99,4.1,15,/art-therapy-based-on-vedic-sciences/
9,Chrysalis: Spirituality & Science United for E...,39,None,8 hour,All Levels,10.99,39.99,4.7,4,/chrysalis-teachings-spiritual-insights-for-cu...


In [112]:
with open ('all_my_data_1288.pkl','wb') as picklefile:
    pickle.dump(all_data, picklefile)

In [114]:
all_data[0][-1]

'/android-using-backendless-to-save-data-in-the-cloud/'

In [115]:
url_list = [all_data[i][-1] for i in range(1288)]

In [117]:
with open ('all_my_data_1288.pkl','wb') as picklefile:
    pickle.dump(all_data, picklefile)

'/tclientdataset1/'

In [120]:
with open ('udemy_course_links.pkl','wb') as picklefile:
    pickle.dump(url_list, picklefile)

In [119]:
url_list

['/android-using-backendless-to-save-data-in-the-cloud/',
 '/fundraising-success-the-art-science-of-major-gifts/',
 '/positive-psychology-habits-and-happiness-certification/',
 '/the-science-of-reiki/',
 '/understanding-financial-malware-and-how-top-banks-protect/',
 '/death-by-powerpoint-to-killer-presentation/',
 '/science-of-goal-setting/',
 '/how-to-read-data-from-a-modbus-rs485-temperature-sensor/',
 '/art-therapy-based-on-vedic-sciences/',
 '/chrysalis-teachings-spiritual-insights-for-curious-seekers/',
 '/seo-schema-tutorial-structured-data-rich-snippets-json-ld/',
 '/understanding-the-art-and-science-of-anger/',
 '/android-using-backendless-to-save-data-in-the-cloud/',
 '/fundraising-success-the-art-science-of-major-gifts/',
 '/positive-psychology-habits-and-happiness-certification/',
 '/the-science-of-reiki/',
 '/understanding-financial-malware-and-how-top-banks-protect/',
 '/death-by-powerpoint-to-killer-presentation/',
 '/science-of-goal-setting/',
 '/how-to-read-data-from-a

In [131]:
# finish well
driver.quit()
os.system('killall chrome')

256